In [73]:
import pandas as pd
#TFIDF - good for contrasting differences.
#Still need to work on BoW to compare similarities.

from sklearn.feature_extraction.text import CountVectorizer

file = r"C:\Users\yotam.twersky\Downloads\Competitor Content.csv"
error_lines_bad = False
df = pd.read_csv(file)
print (df.head(10))
print (df['Company'])
filename = r"C:\Users\yotam.twersky\Downloads\OneDrive_1_1-31-2023\companal1.csv"

      Company                                      Homepage Text
0     Seclore  What If …\nData Carries Its Own\nSecurity & Pr...
1      Citrix  Support any user, anywhere \nMeet the evolving...
2       Axway  Announcing a new kind of API marketplace. Your...
3  GoAnywhere  Simplify and Secure File Transfers with the #1...
4     PreVeil  Simple, encrypted document collaboration and e...
5      Egress  People and technology aren’t a perfect mix.\nW...
6      Egnyte  Secure Enterprise File Sharing\nSTART FREE TRI...
7    Echoworx  Home\nOnly Echoworx customizable encryption of...
8   Kiteworks  \nLeverage content-defined zero trust to unify...
0       Seclore
1        Citrix
2         Axway
3    GoAnywhere
4       PreVeil
5        Egress
6        Egnyte
7      Echoworx
8     Kiteworks
Name: Company, dtype: object


In [78]:


import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

docs=list(df['Homepage Text'])
#instantiate CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(docs)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])
# count matrix 
count_vector=cv.transform(docs) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names() 
#get tfidf vector for first document 
#print the scores 
tfidfdict = {}
complist = []
for i in range(len(df['Company'])):
    seclore_vector=tf_idf_vector[i]
    seclore_df_tfidf = pd.DataFrame(seclore_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
    company = df['Company'][i]
    resultsn = seclore_df_tfidf.sort_values(by=["tfidf"],ascending=False)
    resultsn['Company'] = company
    complist.append(resultsn)
tfidfresult = pd.concat(complist)
tfidfresult.to_csv(filename, index=True, header=True, columns=["tfidf","Company"])

filename = r"C:\Users\yotam.twersky\Downloads\OneDrive_1_1-31-2023\companal2.csv"




    #for row in resultsn.index:
    #    complist.append(row)
    #tfidfdict[company] = complist
#print(tfidfdict)



c:\Users\yotam.twersky\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\yotam.twersky\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [97]:
import copy
import csv
filename = r"C:\Users\yotam.twersky\Downloads\OneDrive_1_1-31-2023\companal2.csv"


print(len(df['Company']))
vocabulary = cv.vocabulary_
bag_of_words = word_count_vector.toarray()

print(vocabulary['what'])



newdict = {}
for key in vocabulary:
    newdict[key] = []
    for i in range(len(df['Company'])):
        newdict[key].append(bag_of_words[i][vocabulary.get(key)])


with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key'] + [df['Company'][i] for i in range(len(df['Company']))])
    for key, value in newdict.items():
        writer.writerow([key] + value)

filename = r"C:\Users\yotam.twersky\Downloads\OneDrive_1_1-31-2023\companal3.csv"

9
1180


In [84]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


sentiment_scores = []
for document in range(len(docs)):
    sentiment_scores.append([
        df['Company'][document],
        sia.polarity_scores(docs[document])["neg"],
        sia.polarity_scores(docs[document])["neu"],
        sia.polarity_scores(docs[document])["pos"],
        sia.polarity_scores(docs[document])["compound"]
    ])

with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Company Name", "Neg", "Neu", "Pos", "Compound"])
    for sentiment_score in sentiment_scores:
        writer.writerow(sentiment_score)


filename = r"C:\Users\yotam.twersky\Downloads\OneDrive_1_1-31-2023\companal4.csv"



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\yotam.twersky\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(docs)

cosine_similarities = cosine_similarity(tfidf_matrix)

cosine_similarities_dict = {}
for i in range(len(docs)):
    for j in range(i+1, len(docs)):
        cosine_similarities_dict[(df['Company'][i], df['Company'][j])] = cosine_similarities[i][j]

print(cosine_similarities_dict)



{('Seclore', 'Citrix'): 0.14990580652419477, ('Seclore', 'Axway'): 0.12413230258819968, ('Seclore', 'GoAnywhere'): 0.1739847232200147, ('Seclore', 'PreVeil'): 0.17771649234300746, ('Seclore', 'Egress'): 0.12265145936923318, ('Seclore', 'Egnyte'): 0.14013983577745398, ('Seclore', 'Echoworx'): 0.16272896210031731, ('Seclore', 'Kiteworks'): 0.18205010401359015, ('Citrix', 'Axway'): 0.2566609771165778, ('Citrix', 'GoAnywhere'): 0.30020038564541807, ('Citrix', 'PreVeil'): 0.2232318793800678, ('Citrix', 'Egress'): 0.12119934458619572, ('Citrix', 'Egnyte'): 0.21946003177150814, ('Citrix', 'Echoworx'): 0.3159430875540551, ('Citrix', 'Kiteworks'): 0.26589385471407323, ('Axway', 'GoAnywhere'): 0.28538406970825264, ('Axway', 'PreVeil'): 0.192352730351954, ('Axway', 'Egress'): 0.12063686574684403, ('Axway', 'Egnyte'): 0.20141779773759022, ('Axway', 'Echoworx'): 0.2971330972196313, ('Axway', 'Kiteworks'): 0.273744965815279, ('GoAnywhere', 'PreVeil'): 0.2665760017735616, ('GoAnywhere', 'Egress'): 0.

In [86]:
with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Company 1", "Company 2", "Cosine Similarity"])
    for indices, score in cosine_similarities_dict.items():
        writer.writerow([indices[0], indices[1], score])